In [ ]:
import json
import os
from dotenv import load_dotenv
from minsearch import Index

load_dotenv()

os.environ["HF_HOME"] = "/run/cache/"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model_id = "microsoft/Phi-3-small-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype="auto",
  
    
    trust_remote_code=True
)
assert torch.cuda.is_available(), "This model needs a GPU to run ..."
device = torch.cuda.current_device()
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id,  trust_remote_code=True)

In [ ]:
documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    

In [ ]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [ ]:
textfields = ["text", "section", "question"]


indobject = Index(text_fields = textfields, keyword_fields = ['course'])
indobject.fit(documents)

In [ ]:
def search(query, boost_dict = {"question": 3}, filter_dict =  {"course":"mlops-zoomcamp"}, num_results = 5 ):

    context = indobject.search(query = query, boost_dict = boost_dict, filter_dict = filter_dict, num_results = num_results
                    )
    
    return context

In [ ]:
def build_prompt(query, related_docs):
    
    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [ ]:
def chat(prompt, generation_args = {}):
    
    messages = [{"role":"user", "content": prompt.strip()}]
    pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device
)
    if not generation_args:
        generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False
    }

    output = pipe(messages, **generation_args)
    print(output)
    
    return output[0]['generated_text']
    

In [ ]:
def rag(query, boost_dict = {"question": 3}, course_filter =  {"course": "data-engineering-zoomcamp"}, generation_args ={}):
    
    context = search(query = query, boost_dict = boost_dict, filter_dict = course_filter)
    
    prompt = build_prompt(query, context)
    
    answer = chat(prompt, generation_args = generation_args)
    
    return answer
    

In [ ]:
query = "How to run spark engine ?"

In [ ]:
rag(query)